# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4, timeoutcontroller
from datetime import datetime
from fmlc.triggering import triggering
from fmlc.baseclasses import eFMU
from fmlc.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
# logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1704217927.5
Main triggered 1704217928.0
Print triggered 1704217928.0
Main triggered 1704217928.5
Main triggered 1704217929.0
Print triggered 1704217929.0


# Controller Base Class (eFMU)

In [4]:
    # Test controller
    testcontroller = testcontroller1() #this controller multiplies the inputs
    # Get all variables
    variables = testcontroller.get_model_variables()
    # Makeup some inputs
    inputs = {}
    for var in variables:
        inputs[var] = random.randint(1,50)
    # Query controller
    print ('Log-message', testcontroller.do_step(inputs=inputs))
    print ('Input', testcontroller.input)
    print ('Output', testcontroller.output)
    print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 9, 'b': 15}
Output {'c': 135}
Output {'c': 135}


# Controller Stack Class (single-thread/multi-thread)

In [5]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'function': testcontroller1, 'sampletime': 1}
controllers['mpc1'] = {'function': testcontroller2, 'sampletime': 'forecast1'}
controllers['control1'] = {'function': testcontroller3, 'sampletime': 'mpc1'}
controllers['forecast2'] = {'function': testcontroller3, 'sampletime': 1}
controllers['forecast3'] = {'function': testcontroller1, 'sampletime': 1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=False, parallel=False, timeout=2)

# Call query_control for 20 seconds.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,time,uid,c,logging
2024-01-02 09:52:09.251397120,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:12.500000000,10,4,1704217932.5,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:14.727994112,10,4,1704217934.727994,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:16.955002624,10,4,1704217936.955003,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:19.181738752,10,4,1704217939.181739,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:21.408254464,10,4,1704217941.408254,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:23.634658816,10,4,1704217943.634659,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:25.861302784,10,4,1704217945.861303,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:28.087682048,10,4,1704217948.087682,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:30.314557184,10,4,1704217950.314557,forecast1,40.0,testcontroller1 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:09.251397120,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:12.500000000,40,10,1704217932.5,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:14.727994112,40,10,1704217934.727994,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:16.955002624,40,10,1704217936.955003,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:19.181738752,40,10,1704217939.181739,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:21.408254464,40,10,1704217941.408254,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:23.634658816,40,10,1704217943.634659,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:25.861302784,40,10,1704217945.861303,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:28.087682048,40,10,1704217948.087682,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:30.314557184,40,10,1704217950.314557,mpc1,400.0,testcontroller2 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:09.251397120,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:12.500000000,400,40,1704217932.5,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:14.727994112,400,40,1704217934.727994,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:16.955002624,400,40,1704217936.955003,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:19.181738752,400,40,1704217939.181739,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:21.408254464,400,40,1704217941.408254,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:23.634658816,400,40,1704217943.634659,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:25.861302784,400,40,1704217945.861303,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:28.087682048,400,40,1704217948.087682,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:30.314557184,400,40,1704217950.314557,control1,16000.0,testcontroller3 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:09.251397120,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:12.500000000,20,4,1704217932.5,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:14.727994112,20,4,1704217934.727994,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:16.955002624,20,4,1704217936.955003,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:19.181738752,20,4,1704217939.181739,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:21.408254464,20,4,1704217941.408254,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:23.634658816,20,4,1704217943.634659,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:25.861302784,20,4,1704217945.861303,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:28.087682048,20,4,1704217948.087682,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:30.314557184,20,4,1704217950.314557,forecast2,80.0,testcontroller3 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:09.251397120,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:12.500000000,30,4,1704217932.5,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:14.727994112,30,4,1704217934.727994,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:16.955002624,30,4,1704217936.955003,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:19.181738752,30,4,1704217939.181739,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:21.408254464,30,4,1704217941.408254,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:23.634658816,30,4,1704217943.634659,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:25.861302784,30,4,1704217945.861303,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:28.087682048,30,4,1704217948.087682,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:30.314557184,30,4,1704217950.314557,forecast3,120.0,testcontroller1 did a computation!


24.751866340637207


## Multi Thread

In [7]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'function': testcontroller1, 'sampletime': 1}
controllers['mpc1'] = {'function': testcontroller2, 'sampletime': 'forecast1'}
controllers['control1'] = {'function': testcontroller3, 'sampletime': 'mpc1'}
controllers['forecast2'] = {'function': testcontroller3, 'sampletime': 1}
controllers['forecast3'] = {'function': testcontroller1, 'sampletime': 1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=False, parallel=True, workers=200, timeout=2)


# Call query_control for 2 minutes. We should expect close to 120 recordings per "first-in-queue" controller.
# In multi thread mod, each call of query_control will trigger a computation for all controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
ctrl_stack.run_query_control_for(120)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,time,uid,c,logging
2024-01-02 09:52:37.052557824,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:40.250000128,10,4,1704217960.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:41.250000128,10,4,1704217961.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:42.250000128,10,4,1704217962.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:52:43.250000128,10,4,1704217963.25,forecast1,40.0,testcontroller1 did a computation!
...,...,...,...,...,...,...
2024-01-02 09:54:36.250000128,10,4,1704218076.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:37.250000128,10,4,1704218077.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:38.250000128,10,4,1704218078.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:39.250000128,10,4,1704218079.25,forecast1,40.0,testcontroller1 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:37.052557824,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:40.250000128,40,10,1704217960.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:41.250000128,40,10,1704217961.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:42.250000128,40,10,1704217962.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:52:43.250000128,40,10,1704217963.25,mpc1,400.0,testcontroller2 did a computation!
...,...,...,...,...,...,...
2024-01-02 09:54:36.250000128,40,10,1704218076.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:37.250000128,40,10,1704218077.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:38.250000128,40,10,1704218078.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:39.250000128,40,10,1704218079.25,mpc1,400.0,testcontroller2 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:37.052557824,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:40.250000128,400,40,1704217960.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:42.250000128,400,40,1704217962.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:44.250000128,400,40,1704217964.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:52:46.250000128,400,40,1704217966.25,control1,16000.0,testcontroller3 did a computation!
...,...,...,...,...,...,...
2024-01-02 09:54:32.250000128,400,40,1704218072.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:34.250000128,400,40,1704218074.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:36.250000128,400,40,1704218076.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:38.250000128,400,40,1704218078.25,control1,16000.0,testcontroller3 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:37.052557824,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:40.250000128,20,4,1704217960.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:42.250000128,20,4,1704217962.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:44.250000128,20,4,1704217964.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:52:46.250000128,20,4,1704217966.25,forecast2,80.0,testcontroller3 did a computation!
...,...,...,...,...,...,...
2024-01-02 09:54:32.250000128,20,4,1704218072.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:34.250000128,20,4,1704218074.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:36.250000128,20,4,1704218076.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:38.250000128,20,4,1704218078.25,forecast2,80.0,testcontroller3 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:52:37.052557824,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:52:40.250000128,30,4,1704217960.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:41.250000128,30,4,1704217961.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:42.250000128,30,4,1704217962.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:52:43.250000128,30,4,1704217963.25,forecast3,120.0,testcontroller1 did a computation!
...,...,...,...,...,...,...
2024-01-02 09:54:36.250000128,30,4,1704218076.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:37.250000128,30,4,1704218077.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:38.250000128,30,4,1704218078.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:39.250000128,30,4,1704218079.25,forecast3,120.0,testcontroller1 did a computation!


121.82299375534058


In [8]:
# refresh the mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'function': testcontroller1, 'sampletime': 1}
controllers['mpc1'] = {'function': testcontroller2, 'sampletime': 'forecast1'}
controllers['control1'] = {'function': testcontroller3, 'sampletime': 'mpc1'}
controllers['forecast2'] = {'function': testcontroller3, 'sampletime': 1}
controllers['forecast3'] = {'function': testcontroller1, 'sampletime': 1}

#Adding timeoutcontroller
controllers['timeout'] = {'function': timeoutcontroller, 'sampletime': 5}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=False, parallel=True, timeout=2, workers=200)
print("done initializing")

# Call query_control for 20 seconds. We should expect a bunch of timeouts.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

done initializing


/usr/local/lib/python3.8/dist-packages/fmlc/stackedclasses.py:370: Warning: ERROR: Controller "timeout": 

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/fmlc/stackedclasses.py", line 359, in run_controller_queue
    p.result(self.timeout)
  File "/usr/lib/python3.8/concurrent/futures/_base.py", line 441, in result
    raise TimeoutError()
concurrent.futures._base.TimeoutError

  warnings.warn(f'ERROR: Controller "{name}": {e}\n\n{traceback.format_exc()}', Warning)


,a,b,time,uid,c,logging
2024-01-02 09:54:41.926224384,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:54:45.250000128,10,4,1704218085.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:46.250000128,10,4,1704218086.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:47.250000128,10,4,1704218087.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:48.250000128,10,4,1704218088.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:49.250000128,10,4,1704218089.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:50.250000128,10,4,1704218090.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:51.250000128,10,4,1704218091.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:52.250000128,10,4,1704218092.25,forecast1,40.0,testcontroller1 did a computation!
2024-01-02 09:54:53.250000128,10,4,1704218093.25,forecast1,40.0,testcontroller1 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:54:41.926224384,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:54:45.250000128,40,10,1704218085.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:46.250000128,40,10,1704218086.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:47.250000128,40,10,1704218087.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:48.250000128,40,10,1704218088.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:49.250000128,40,10,1704218089.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:50.250000128,40,10,1704218090.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:51.250000128,40,10,1704218091.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:52.250000128,40,10,1704218092.25,mpc1,400.0,testcontroller2 did a computation!
2024-01-02 09:54:53.250000128,40,10,1704218093.25,mpc1,400.0,testcontroller2 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:54:41.926224384,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:54:45.250000128,400,40,1704218085.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:47.250000128,400,40,1704218087.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:49.250000128,400,40,1704218089.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:51.250000128,400,40,1704218091.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:53.250000128,400,40,1704218093.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:55.250000128,400,40,1704218095.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:57.250000128,400,40,1704218097.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:54:59.250000128,400,40,1704218099.25,control1,16000.0,testcontroller3 did a computation!
2024-01-02 09:55:01.250000128,400,40,1704218101.25,control1,16000.0,testcontroller3 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:54:41.926224384,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:54:45.250000128,20,4,1704218085.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:47.250000128,20,4,1704218087.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:49.250000128,20,4,1704218089.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:51.250000128,20,4,1704218091.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:53.250000128,20,4,1704218093.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:55.250000128,20,4,1704218095.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:57.250000128,20,4,1704218097.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:54:59.250000128,20,4,1704218099.25,forecast2,80.0,testcontroller3 did a computation!
2024-01-02 09:55:01.250000128,20,4,1704218101.25,forecast2,80.0,testcontroller3 did a computation!


,a,b,time,uid,c,logging
2024-01-02 09:54:41.926224384,NaN,NaN,NaN,NaN,NaN,Initialize
2024-01-02 09:54:45.250000128,30,4,1704218085.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:46.250000128,30,4,1704218086.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:47.250000128,30,4,1704218087.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:48.250000128,30,4,1704218088.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:49.250000128,30,4,1704218089.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:50.250000128,30,4,1704218090.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:51.250000128,30,4,1704218091.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:52.250000128,30,4,1704218092.25,forecast3,120.0,testcontroller1 did a computation!
2024-01-02 09:54:53.250000128,30,4,1704218093.25,forecast3,120.0,testcontroller1 did a computation!


,time,uid,a,logging
2024-01-02 09:54:41.926224384,NaN,NaN,NaN,Initialize
2024-01-02 09:54:45.250000128,1704218085.25,timeout,seen due to late computation,Compute ok
2024-01-02 09:54:50.250000128,1704218090.25,timeout,seen due to late computation,Compute ok
2024-01-02 09:54:55.250000128,1704218095.25,timeout,seen due to late computation,Compute ok
2024-01-02 09:55:00.250000128,1704218100.25,timeout,seen due to late computation,Compute ok
2024-01-02 09:55:05.250000128,1704218105.25,timeout,seen due to late computation,Compute ok


22.648305416107178
